# BLEU Score

Code for the post [Understanding the BLEU Score for Seq2Seq Model Evaluation](https://deconvoluteai.com/blog/bleu-score).

If you have any questions, feel free to reach out to me at [Contact](http://deconvoluteai.com/contact).

In [1]:
# !pip install numpy
from collections import Counter
import math
import numpy as np

# Preprocessing

The BLEU score calculation works with sentences as the basic unit. For each sentence, we compare it against one or more reference translations.

Given these requirements, we'll use a list of lists of strings, where each word is an element of the inner list, and each list represents a sentence. 
Additionally, we will case-fold all strings and remove punctuation. 

To construct this list of lists of words, we use this function to convert a list of sentences.

In [2]:
def preprocess_text(sentences: list[str]) -> list[list[str]]:
    """Splits a list of strings, for example a list of sentences, into a list of list of strings, and removes punctionation, and case folds.
    The input could be a list of at least one sentence as a string.
    
    Args:
        sentences: A list of strings or a list of lists containing strings.

    Returns:
        A list of lists where each inner list contains words from the corresponding input string.
    """
    
    if not type(sentences) == list:
        raise ValueError("Expecting a list of strings as input.")
    return [["".join(char.lower() for char in word if char.isalnum()) for word in sentence.split()] for sentence in sentences]

# Tests
assert (preprocess_text(["Hello, this is some text."]) == [['hello', 'this', 'is', 'some', 'text']])
assert (preprocess_text(["Hello, this is some text.", "This is a second sentence!"]) == [['hello', 'this', 'is', 'some', 'text'], ['this', 'is', 'a', 'second', 'sentence']])

# n-gram Precision

Using the preprocessing function, we can ensure that our inputs are always in the correct format. 
To calculate the n-gram precision, we define a function that takes a `candidate` sentence and a list of `references`, and returns the result as a tuple containing the numerator and denominator. 
This makes it easier to verify the results.

In [3]:
def n_gram_precision(candidate: list[str], references: list[list[str]], n: int) -> tuple[int, int]:
    """Calculates the n-gram precision for a candidate given a list of references.
    
    Args:
        candidate: A list of words that make up a sentence.
        references: A list of reference sentences, each being a list of words.
        n: The parameter for the n-gram.

    Returns:
        The clipped n-gram count and the total n-gram count.
    """

    if n < 1:
        raise ValueError("n must be greater or equal 1.")

    # Count n-grams in candidate
    candidate_n_grams = Counter([tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)])

    # Count n-grams in references and take the maximum counts for each n-gram
    max_reference_n_grams = Counter()
    for reference in references:
        reference_n_grams = Counter([tuple(reference[i:i+n]) for i in range(len(reference)-n+1)])
        for n_gram in reference_n_grams:
            max_reference_n_grams[n_gram] = max(max_reference_n_grams[n_gram], reference_n_grams[n_gram])

    # Clip counts
    clipped_counts = {ng: min(count, max_reference_n_grams[ng]) for ng, count in candidate_n_grams.items()}
    
    return sum(clipped_counts.values()), sum(candidate_n_grams.values())

We can test this function with the examples from the paper.

In [4]:
# Example 1
candidate11 = preprocess_text(["It is a guide to action which ensures that the military always obeys the commands of the party."])
candidate12 = preprocess_text(["It is to insure the troops forever hearing the activity guidebook that party direct."])
references1 = preprocess_text(["It is a guide to action that ensures that the military will forever heed Party commands.", 
                               "It is the guiding principle which guarantees the military forces always being under the command of the Party.",
                               "It is the practical guide for the army always to heed the directions of the party."])

# Example 2
candidate21 = preprocess_text(["the the the the the the the."])
references2 = preprocess_text(["The cat is on the mat.", "The cat is on the mat."])

# Tests
assert(n_gram_precision(candidate11[0], references1, 1) == (17,18))
assert(n_gram_precision(candidate12[0], references1, 1) == (8,14))
assert(n_gram_precision(candidate21[0], references2, 1) == (2,7))


# Brevity Penalty

The final part we need is the brevity penalty calculation. 
This function penalizes shorter candidate translations to avoid rewarding incomplete translations.

In [5]:
def brevity_penalty(candidate: list[str], references: list[list[str]]) -> float:
    """Calculates the brevity penalty for a candidate sentence against reference translations.
    
    Args:
        candidate: A list of words from the candidate sentence.
        references: A list of lists of words from the reference translations.
    
    Returns:
        The brevity penalty score.
    """
    
    c = len(candidate)
    r = min(len(reference) for reference in references)
    
    if c > r:
        return 1
    else:
        return math.exp(1 - r / c)

# Tests
candidate1 = ["A", "test"]
references1 = [["A", "test"], ["Another", "test"] ]
references2 = [["A", "test", "hello"], ["Another", "test", "longer", "better?"], ["And", "another", "test", "longer", "worse!"] ]

assert(brevity_penalty(candidate1, references1) == 1.0)
assert(brevity_penalty(candidate1, references2) == math.exp(-0.5))

# Final BLEU Score Calculation

Using the three components (n-gram precision, brevity penalty, and geometric mean), we can now calculate the final BLEU score. 
This function combines all the steps and provides a comprehensive evaluation of the candidate translation against the reference translations.

In [6]:
def bleu_score_sentence(candidate_seq: str, references_seqs: list[str], max_n: int=4) -> float:
    """Calculates the BLEU score with uniform weights for a candidate given a list of references.
    
    Args:
        candidate: A string with the candidate sentence.
        references: A list of strings with reference sentences, one string for each sentence.
        max_n: The number up to which n-grams should be calculated.
    
    Returns:
        BLEU Score     
    """

    candidate_seq: list[str] = preprocess_text([candidate_seq])[0]
    references_seqs: list[list[str]] = preprocess_text(references_seqs)

    precisions = []

    for n in range(1, max_n+1):
        p_n, total = n_gram_precision(candidate_seq, references_seqs, n)
        precisions.append(p_n / total if total > 0 else 0)
    
    if all(p == 0 for p in precisions):
        return 0
    
    geometric_mean = np.exp(np.mean([math.log(p) for p in precisions if p > 0]))
    bp = brevity_penalty(candidate_seq, references_seqs)

    return bp * geometric_mean

# Using the Score

In [7]:
# Example 1
candidate_seq = "This is a test."
reference = ["This is a test.", "This is a test."]
print(bleu_score_sentence(candidate_seq, reference, max_n=4))

# Example 2
candidate11 = "It is a guide to action which ensures that the military always obeys the commands of the party."
candidate12 = "It is to insure the troops forever hearing the activity guidebook that party direct."
references1 = [
    "It is a guide to action that ensures that the military will forever heed Party commands.", 
    "It is the guiding principle which guarantees the military forces always being under the command of the Party.",
    "It is the practical guide for the army always to heed the directions of the party."
    ]
print(bleu_score_sentence(candidate11, references1, 4))
print(bleu_score_sentence(candidate12, references1, 4))

# Example 3
candidate21 = "the the the the the the the."
references2 = ["The cat is on the mat.", "The cat is on the mat."]

print(bleu_score_sentence(candidate21, references2, 4))

1.0
0.5045666840058485
0.18174699151949172
0.2857142857142857
